In [1]:
# coding: utf-8

# In[7]:

import sys,os
import numpy as np
import dicom
import glob
import matplotlib
# matplotlib.use('Agg')
from matplotlib import pyplot as plt
import cv2
import pandas as pd
import time
from sklearn import cross_validation
from lightgbm.sklearn import LGBMRegressor
#from Vgg19 import VGG19
from resnet50 import ResNet50
from keras.preprocessing import image
#from imagenet_utils import preprocess_input
#from PIL import Image
#from resizeimage import resizeimage

/a/data/mango/environments/newtf/lib/python2.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)
/a/data/mango/environments/newtf/lib/python2.7/site-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:28: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))
/a/data/mango/environments/newtf/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this mo

In [2]:
import errno
def make_sure_path_exists(path):
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise
EXPERIMENT_NUMBER = '005' 

#Put here the path where you downloaded all kaggle data
DATA_PATH='data/'

# Path and variables
STAGE1_LABELS=DATA_PATH + 'stage1_labels.csv'
STAGE1_SAMPLE_SUBMISSION=DATA_PATH + 'stage1_sample_submission.csv'
STAGE1_FOLDER=DATA_PATH + 'stage1/'
FEATURE_FOLDER=DATA_PATH + 'features/features' + EXPERIMENT_NUMBER + '/'
SUBMIT_OUTPUT='submit' + EXPERIMENT_NUMBER + '.csv'

make_sure_path_exists(FEATURE_FOLDER)

In [3]:
# Timer class
class Timer(object):
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def start(self):
        self.start = time.clock()

    def stop(self):
        self.end = time.clock()
        self.interval = self.end - self.start


In [4]:
def get_3d_data(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key=lambda x: int(x.InstanceNumber))
    return np.stack([s.pixel_array for s in slices])

In [5]:
def get_data_id(path, plot_data=False):
    sample_image = get_3d_data(path)

    sample_image[sample_image == -2000] = 0
    # downsample
    sample_image = sample_image[:94]
    print "sample_image shape:"
    print sample_image.shape
    #if plot_data:
    #    f, plots = plt.subplots(4, 5, sharex='col', sharey='row', figsize=(10, 8))

    batch = []
    cnt = 0
    dx = 40
    ds = 512
    for i in range(0, sample_image.shape[0] - 3, 3):
        tmp = []
        for j in range(3):
            img = sample_image[i + j]
            img = 255.0 / np.amax(img) * img
            img = cv2.equalizeHist(img.astype(np.uint8))
            img = img.astype(np.uint8)
            img = img[dx: ds - dx, dx: ds - dx]
            img = cv2.resize(img, (224, 224))
            #img = resizeimage.resize_crop(img, [224, 224])
            #img = np.resize(img, (224,224))
            tmp.append(img)

        tmp = np.array(tmp)
        tmp = np.swapaxes(tmp,0,2) #we need to make "color" dimension last I think
        batch.append(np.array(tmp))

#         if plot_data:
#             if cnt < 20:
#                 plots[cnt // 5, cnt % 5].axis('off')
#                 plots[cnt // 5, cnt % 5].imshow(tmp[0,:,:], cmap='gray')
#             cnt += 1

#     if plot_data: plt.show()
        
    batch = np.array(batch, dtype='int')
    return batch



In [6]:
def get_model():
    model = ResNet50(weights='imagenet', include_top=False)
    return model

In [25]:
def calc_feature(verbose=False):
    model = get_model()
    for folder in glob.glob(STAGE1_FOLDER+'*'):
        foldername = os.path.basename(folder)
        if os.path.isfile(FEATURE_FOLDER+foldername+'.npy'):
            if verbose:print('Features in %s already computed' %(FEATURE_FOLDER+foldername))
            continue
        batch = get_data_id(folder)
        if verbose:
            print ("Batch size:")
            print (batch.shape)
        feats = []
        for i in range(batch.shape[0]):
            temp = batch[i]
            temp = np.expand_dims(temp, axis = 0)
            feat = model.predict(temp, batch_size = 52, verbose = 1)
            print "feat.shape"
            print feat.shape
            feats.append(feat)
        print "feats.shape"
        print len(feats)

In [7]:
def calc_features(verbose=False):
    model = get_model()
    for folder in glob.glob(STAGE1_FOLDER+'*'):
        foldername = os.path.basename(folder)
        if os.path.isfile(FEATURE_FOLDER+foldername+'.npy'):
            if verbose: print("Features in %s already computed" % (FEATURE_FOLDER+foldername))
            continue
        batch = get_data_id(folder)
        print ("batch shape")
        print batch.shape
        if verbose:
            print("Batch size:")
            print(batch.shape)
        
        feats = model.predict(batch, batch_size=52, verbose=1)
        print foldername
        print feats.shape

        if verbose:
            print(feats.shape)
            print("Saving features in %s" % (FEATURE_FOLDER+foldername))
        np.save(FEATURE_FOLDER+foldername, feats)

In [8]:
        
#%%time
# Calculate features
calc_features(verbose=False)

In [9]:
# add LSTM to model
from keras.models import Sequential
from keras.layers import LSTM, Dense
#import numpy as np

# feats shape (31, 1,1,2048) for every patient
# each feat for every patient with shape (1,1,1,2048), so input_dim = 4
# timesteps, total 31 feats for every patient
# patient number:
def train_lstm():
    data_dim = 1
    timesteps = 1 
    num_classes = 2

    # expected input data shape: (batch_size, timesteps, data_dim)
    model = Sequential()
    model.add(LSTM(32, return_sequences=False,
                   input_shape=(1, 2048)))  # returns a sequence of vectors of dimension 32
    #model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
    #model.add(LSTM(32))  # return a single vector of dimension 32
    model.add(Dense(1, activation='softmax')) #  is number of classes

    model.compile(loss='mean_squared_error',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    df  = pd.read_csv(STAGE1_LABELS)
    y = df['cancer'].as_matrix()
    print y.shape
    x =  np.array([np.mean(np.load(FEATURE_FOLDER+'%s.npy' % str(id)), axis=0).flatten() for id in df['id'].tolist()])
    x = x.reshape(x.shape[0], 1, x.shape[1])
    x_train, x_val, y_train, y_val = cross_validation.train_test_split(x, y, random_state=42, stratify=y,
                                                                       test_size=0.20)
    print x_train.shape
    print y_train.shape
    print x_val.shape
    print y_val.shape

    model.fit(x_train, y_train,
              batch_size=64, epochs=1,
              validation_data=(x_val, y_val))
    return model

In [10]:
def compute_training_lstm(verbose=True):
    with Timer() as t:
        model = train_lstm()
    if verbose: print("Training took %.03f sec.\n" % t.interval)
    return model

In [16]:
def compute_prediction_lstm(model, verbose=True):    
    df = pd.read_csv(STAGE1_SAMPLE_SUBMISSION)
    x = np.array([np.mean(np.load((FEATURE_FOLDER+'%s.npy') % str(id)), axis=0).flatten() for id in df['id'].tolist()])
    x = x.reshape(x.shape[0], 1, x.shape[1])
    with Timer() as t:
        pred = model.predict(x)
    if verbose: print("Prediction took %.03f sec.\n" % t.interval)
    df['cancer'] = pred
    return df

In [17]:
def save_results(df):
    df.to_csv(SUBMIT_OUTPUT, index=False)

In [18]:
model = compute_training_lstm(verbose=True)

(1397,)
(1117, 1, 2048)
(1117,)
(280, 1, 2048)
(280,)
Train on 1117 samples, validate on 280 samples
Epoch 1/1
1117/1117 [==============================] - 0s - loss: 0.7413 - acc: 0.2587 - val_loss: 0.7393 - val_acc: 0.2607
Training took 2.710 sec.



In [19]:
df = compute_prediction_lstm(model)
print("Results:")
df.head()
save_results(df)

Prediction took 0.230 sec.

Results:
